In [1]:
import os
import json
from tqdm import tqdm

import pickle

In [2]:
## nodes: items, brands
## item features: title, description, category, price, img
## item edges: also_viewed, buy_after_viewing, also_bought, bought_together
## brand features: name

In [3]:
raw_data_dir="/home/ec2-user/quic-efs/user/bowenjin/llm-graph-plugin/data/raw_data/amazon"
save_dir="/home/ec2-user/quic-efs/user/bowenjin/llm-graph-plugin/data/processed_data/amazon"

In [4]:
## read raw data files

def read_json_lines(file, id_key):
    data = {}
    with open(file) as f:
        readin = f.readlines()
        for line in tqdm(readin):
            #tmp = json.loads(line)
            tmp = eval(line)
            data[tmp[id_key]] = tmp
    return data

item_raw_data = read_json_lines(os.path.join(raw_data_dir, 'metadata.json'), 'asin')

100%|██████████| 9430088/9430088 [13:44<00:00, 11442.15it/s] 


In [5]:
list(item_raw_data.keys())[:5]

['0001048791', '0000143561', '0000037214', '0000032069', '0000031909']

In [6]:
item_raw_data['0000143561']

{'asin': '0000143561',
 'categories': [['Movies & TV', 'Movies']],
 'description': '3Pack DVD set - Italian Classics, Parties and Holidays.',
 'title': 'Everyday Italian (with Giada de Laurentiis), Volume 1 (3 Pack): Italian Classics, Parties, Holidays',
 'price': 12.99,
 'salesRank': {'Movies & TV': 376041},
 'imUrl': 'http://g-ecx.images-amazon.com/images/G/01/x-site/icons/no-img-sm._CB192198896_.gif',
 'related': {'also_viewed': ['B0036FO6SI',
   'B000KL8ODE',
   '000014357X',
   'B0037718RC',
   'B002I5GNVU',
   'B000RBU4BM'],
  'buy_after_viewing': ['B0036FO6SI',
   'B000KL8ODE',
   '000014357X',
   'B0037718RC']}}

In [7]:
related_name = set()
for itt in tqdm(item_raw_data):
    if "related" in item_raw_data[itt]:
        for kk in item_raw_data[itt]['related'].keys():
            related_name.add(kk)
print(related_name)

  0%|          | 0/9430088 [00:00<?, ?it/s]

100%|██████████| 9430088/9430088 [00:13<00:00, 712206.69it/s]

{'bought_together', 'buy_after_viewing', 'also_bought', 'also_viewed'}


In [8]:
## construct node dictionary
## item features: title, description, category, price, img
## item edges: also_viewed, buy_after_viewing, also_bought, bought_together
## item neighbors: item, brand

## brand features: name
## brand neighbors: item

item_nodes = {}
brand_nodes = {}

brand_name2id = {}

for item_id in tqdm(item_raw_data):
    
    # brand nodes
    if 'brand' in item_raw_data[item_id] and item_raw_data[item_id]['brand'] != '':
        if item_raw_data[item_id]['brand'] not in brand_name2id:
            idd = f'brand_{len(brand_nodes)}'
            brand_name2id[item_raw_data[item_id]['brand']] = idd
            brand_nodes[idd] = {'features': {}, 'neighbors': {}}
            brand_nodes[idd]['features']['name'] = item_raw_data[item_id]['brand']
            brand_nodes[idd]['neighbors']['item'] = [item_id]
        else:
            brand_nodes[brand_name2id[item_raw_data[item_id]['brand']]]['neighbors']['item'].append(item_id)

    # item nodes
    item_nodes[item_id] = {'features': {}, 'neighbors': {}}
    ## add features
    item_nodes[item_id]['features']['title'] = item_raw_data[item_id]['title'] if "title" in item_raw_data[item_id] else ''
    item_nodes[item_id]['features']['description'] = item_raw_data[item_id]['description'] if "description" in item_raw_data[item_id] else ''
    item_nodes[item_id]['features']['price'] = item_raw_data[item_id]['price'] if "price" in item_raw_data[item_id] else ''
    item_nodes[item_id]['features']['img'] = item_raw_data[item_id]['imUrl'] if "imUrl" in item_raw_data[item_id] else ''
    item_nodes[item_id]['features']['category'] = [','.join(c) for c in item_raw_data[item_id]['categories']] if "categories" in item_raw_data[item_id] else []
    ## add neighbors
    if 'related' in item_raw_data[item_id]:
        item_nodes[item_id]['neighbors']['also_viewed_item'] = item_raw_data[item_id]['related']['also_viewed'] if "also_viewed" in item_raw_data[item_id]['related'] else []
        item_nodes[item_id]['neighbors']['buy_after_viewing_item'] = item_raw_data[item_id]['related']['buy_after_viewing'] if "buy_after_viewing" in item_raw_data[item_id]['related'] else []
        item_nodes[item_id]['neighbors']['also_bought_item'] = item_raw_data[item_id]['related']['also_bought'] if "also_bought" in item_raw_data[item_id]['related'] else []
        item_nodes[item_id]['neighbors']['bought_together_item'] = item_raw_data[item_id]['related']['bought_together'] if "bought_together" in item_raw_data[item_id]['related'] else []
    else:
        item_nodes[item_id]['neighbors']['also_viewed_item'] = []
        item_nodes[item_id]['neighbors']['buy_after_viewing_item'] = []
        item_nodes[item_id]['neighbors']['also_bought_item'] = []
        item_nodes[item_id]['neighbors']['bought_together_item'] = []
    item_nodes[item_id]['neighbors']['brand'] = [brand_name2id[item_raw_data[item_id]['brand']]] if ('brand' in item_raw_data[item_id] and item_raw_data[item_id]['brand'] != '') else []

# make the edges bidirectional
for item_id in tqdm(item_nodes):
    for rel in ['also_viewed_item', 'also_bought_item', 'bought_together_item']:
        for nid in item_nodes[item_id]['neighbors'][rel]:
            if nid not in item_nodes:
                item_nodes[item_id]['neighbors'][rel].remove(nid)
                continue
            if item_id not in item_nodes[nid]['neighbors'][rel]:
                item_nodes[nid]['neighbors'][rel].append(item_id)

  0%|          | 0/9430088 [00:00<?, ?it/s]

100%|██████████| 9430088/9430088 [27:19<00:00, 5750.16it/s] 


In [9]:
## save graph
#pickle.dump({
#    'item_nodes': item_nodes,
#    'brand_nodes': brand_nodes,
#}, open(os.path.join(save_dir, 'graph.pkl'),"wb"))

json.dump({
    'item_nodes': item_nodes,
    'brand_nodes': brand_nodes,
}, open(os.path.join(save_dir, 'graph.json'),"w"), indent=4)